## 1. Importing the Required Libraries

In [1]:
import nltk
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer, WordNetLemmatizer
from nltk.corpus import stopwords
import re 
import pandas as pd

## 2. Downloading Required NLTK Resources


In [30]:
nltk.download('punkt_tab')
nltk.download('stopwords')

[nltk_data] Downloading package punkt_tab to
[nltk_data]     C:\Users\berka\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\berka\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

## 3. Loading the Dataset

In [2]:
data = pd.read_csv("tripadvisor_hotel_reviews.csv")
data.head()

,Review,Rating
0,nice hotel expensive parking got good deal sta...,4
1,ok nothing special charge diamond member hilto...,2
2,nice rooms not 4* experience hotel monaco seat...,3
3,"unique, great stay, wonderful time hotel monac...",5
4,"great stay great stay, went seahawk game aweso...",5


In [3]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 109 entries, 0 to 108
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   Review  109 non-null    object
 1   Rating  109 non-null    int64 
dtypes: int64(1), object(1)
memory usage: 1.8+ KB


## 4. Inspecting the First Review

In [7]:
data["Review"][0]

'nice hotel expensive parking got good deal stay hotel anniversary, arrived late evening took advice previous reviews did valet parking, check quick easy, little disappointed non-existent view room room clean nice size, bed comfortable woke stiff neck high pillows, not soundproof like heard music room night morning loud bangs doors opening closing hear people talking hallway, maybe just noisy neighbors, aveda bath products nice, did not goldfish stay nice touch taken advantage staying longer, location great walking distance shopping, overall nice experience having pay 40 parking night,  '

## 5. Converting Text to Lowercase

In [8]:
data["review_lowercase"] = data["Review"].str.lower()
data.head()

,Review,Rating,review_lowercase
0,nice hotel expensive parking got good deal sta...,4,nice hotel expensive parking got good deal sta...
1,ok nothing special charge diamond member hilto...,2,ok nothing special charge diamond member hilto...
2,nice rooms not 4* experience hotel monaco seat...,3,nice rooms not 4* experience hotel monaco seat...
3,"unique, great stay, wonderful time hotel monac...",5,"unique, great stay, wonderful time hotel monac..."
4,"great stay great stay, went seahawk game aweso...",5,"great stay great stay, went seahawk game aweso..."


## 6. Removing Stopwords (Except “not”)

In [9]:
en_stopwords = stopwords.words("english")

In [10]:
en_stopwords.remove("not")

## 7. Removing Stopwords from the Text

In [14]:
data["review_no_stopwords"] = data["review_lowercase"].apply(lambda x: ' '.join([word for word in x.split() if word not in en_stopwords]))

In [15]:
data["review_no_stopwords"][0]

'nice hotel expensive parking got good deal stay hotel anniversary, arrived late evening took advice previous reviews valet parking, check quick easy, little disappointed non-existent view room room clean nice size, bed comfortable woke stiff neck high pillows, not soundproof like heard music room night morning loud bangs doors opening closing hear people talking hallway, maybe noisy neighbors, aveda bath products nice, not goldfish stay nice touch taken advantage staying longer, location great walking distance shopping, overall nice experience pay 40 parking night,'

## 8. Replacing Special Characters (Punctuation Cleaning)

In [19]:
data['review_no_stopwords_no_punct'] = data.apply(
    lambda x: re.sub(r'\*', 'star', x['review_no_stopwords']), axis=1
)

In [20]:
data['review_no_stopwords_no_punct'] = data['review_no_stopwords_no_punct'].apply(
    lambda x: re.sub(r'[^\w\s]', '', x)
)

## 9. Tokenization – Splitting Text into Words

In [21]:
data['tokenized'] = data.apply(
lambda x: word_tokenize(x['review_no_stopwords_no_punct']), axis=1)

## 10. Stemming – Reducing Words to Root Forms

In [22]:
ps = PorterStemmer()
data["stemmed"] = data["tokenized"].apply(lambda tokens: [ps.stem(token) for
token in tokens])


## 11. Lemmatization – Meaningful Root Forms

In [23]:
lemmatizer = WordNetLemmatizer()
data["lemmatized"] = data["tokenized"].apply(lambda tokens:
[lemmatizer.lemmatize(token) for token in tokens])

## 12. Building a Clean Token List

In [24]:
tokens_clean = sum(data['lemmatized'], [])

## 13. Creating n-Grams

In [25]:
unigrams = pd.Series(nltk.ngrams(tokens_clean, 1)).value_counts()

In [28]:
print(unigrams.head(10))

(hotel,)       292
(room,)        275
(great,)       126
(not,)         122
(stay,)         95
(staff,)        90
(nt,)           81
(seattle,)      79
(location,)     78
(good,)         76
Name: count, dtype: int64


In [26]:
bigrams = pd.Series(nltk.ngrams(tokens_clean, 2)).value_counts()

In [29]:
print(bigrams.head(10))

(great, location)      24
(space, needle)        21
(hotel, monaco)        16
(great, hotel)         12
(staff, friendly)      12
(great, view)          12
(pike, place)          12
(walking, distance)    10
(location, great)      10
(pike, market)          9
Name: count, dtype: int64


In [27]:
ngrams_4 = pd.Series(nltk.ngrams(tokens_clean, 4)).value_counts()